In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unrar x '/content/drive/My Drive/COMP4211/data_regression2_3.rar'

Mounted at /content/drive

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/My Drive/COMP4211/data_regression2_3.rar

Creating    data                                                      OK
Extracting  data/meta.csv                                                  0%  OK 
Extracting  data/meta_SP500.csv                                            0%  OK 
Extracting  data/minmax_SP500.pkl                                          0%  OK 
Extracting  data/NASDAQ.csv                                                0%  OK 
Extracting  data/NASDAQ_preprocess.csv                                     0%  OK 
Extracting  data/onehot_encoder.pkl                                        0%  OK 
Extracting  data/onehot_encoder_sp500.pkl                                  0%  OK 
Creating    data/price                                                OK
Extracting  data/price/A.csv       

In [2]:
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.layers import Concatenate,Embedding ,Dense ,Input,LSTM,Permute,Softmax,Lambda,Flatten,GRU,Dropout,BatchNormalization, Normalization, Attention, Bidirectional, Masking, TimeDistributed, AveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.metrics import F1Score, Accuracy
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
!pip install tensorflow-addons
!pip install scikit-learn --upgrade
import tensorflow_addons as tfa
from tensorflow.keras.losses import BinaryCrossentropy
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import h5py
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
class DataGenerator(Sequence):

    def __init__(self, meta_df, batch_size=64, use_full=False, input_shape=(20,6), label_shape=(20,), minmax=None, **kwargs):
        super().__init__(**kwargs)
        self.ds_pointer = 0
        self.batch_size = batch_size
        self.input_shape=input_shape
        self.use_full = use_full
        self.meta_df = meta_df
        self.minmax = minmax
        self.data = self.load_file()
        self.input_buffer = np.zeros((batch_size,input_shape[0],input_shape[1]))
        self.label_buffer = np.zeros((batch_size,label_shape[0]))
        self.reset_pointer()

    def load_file(self):
        result = {}
        for root, _, files in os.walk('data/train_data'):
            for file in files:
                if f'.h5' in file:
                    symbol = file.split('.')[0]
                    df = pd.read_csv(f'data/price/{symbol}.csv')
                    result[symbol] = h5py.File(f'data/train_data/{symbol}.h5', 'r', rdcc_nbytes=100*1024**2, rdcc_nslots=1e4)
        return result

    def reset_pointer(self, seed = 1314):
        self.meta_df = self.meta_df.sample(frac=1, random_state=seed)
        self.ds_pointer = 0

    def __len__(self):
        if self.use_full:
            return len(self.meta_df)// self.batch_size
        else:
            return len(self.meta_df)// self.batch_size// 10

    def num_labels(self):
        return len(self.label_name)

    def upsample(self):
      # Find the maximum count of labels in the DataFrame
      max_count = self.meta_df['Gloss'].value_counts().max()

      # Define a function to resample and reset index for each group
      def resample_and_reset_index(group):
          resampled_group = group.sample(max_count, replace=True)
          return resampled_group

      # Apply the function to each group and concatenate them back into a single DataFrame
      self.meta_df = self.meta_df.groupby('Label').apply(resample_and_reset_index).reset_index(drop=True)

    def __getitem__(self, index):
        import os
        import numpy as np
        if self.ds_pointer + self.batch_size >= len(self.meta_df):
            self.reset_pointer()
            return self.__getitem__(index)
        to_get = self.meta_df.iloc[self.ds_pointer:self.ds_pointer+ self.batch_size]
        to_get_index = to_get['Index'].to_numpy()
        to_get_symbol = to_get['Symbol'].to_numpy()

        for i, symbol in enumerate(to_get_symbol):
            to_fetch = self.data[symbol]
            self.input_buffer[i, :, :] = to_fetch['data'][to_get_index[i]]
            self.label_buffer[i, :] = to_fetch['label'][to_get_index[i]]

        batch_x = self.input_buffer
        batch_y = self.label_buffer
        if self.minmax is not None:
          batch_x = batch_x.reshape(-1,self.input_shape[1])
          batch_x = self.minmax.transform(batch_x)
          batch_x = batch_x.reshape(-1,self.input_shape[0],self.input_shape[1])
          batch_y = batch_y.reshape(-1,self.input_shape[1])
          batch_y = self.minmax.transform(batch_y)
          batch_y = batch_y.reshape(-1,self.input_shape[0],self.input_shape[1])

        self.ds_pointer+=self.batch_size
        return batch_x, batch_y

In [43]:
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def build_model(time_series,num_of_features, scaler=None, init_neuron=16, learning_rate=0.0001):
    K.clear_session()
    input_ = Input(shape=(time_series,num_of_features), name='Input')
    if scaler is not None:
      mean = scaler.mean_
      print(mean[0:5])
      var = scaler.var_
      norm = Normalization(mean=mean, variance=var,name='Normalization')
      norm = norm(input_)
      input_dense = norm
    else:
      input_dense = input_

    # Define GRU layer
    encoder = Bidirectional(GRU(init_neuron, return_state=True, return_sequences=True, name='Encoder'))
    encoder_outputs, forward_h, backward_h = encoder(input_dense)

    state_h = Concatenate()([forward_h, backward_h])

    # # Define attention layer
    attention = Attention(name='Attention')
    context_vector = attention([encoder_outputs, state_h])

    # Concatenate context vector and encoder outputs
    concat_layer = Concatenate(axis=-1, name='Concatenate')
    decoder_combined_context = concat_layer([context_vector, input_dense])

    # Define decoder
    decoder_gru = GRU(init_neuron * 2, return_sequences=True, name='Decoder')

    # Pass the concatenated input through the decoder
    decoder_output = decoder_gru(decoder_combined_context, initial_state=state_h)
    output = AveragePooling1D(pool_size=8, strides=10)(decoder_output)
    output = TimeDistributed(Dense(1))(output)

    model = Model(inputs = input_ , outputs = output)
    adam_optimizer = Adam(learning_rate=learning_rate, weight_decay=1e-7)
    model.compile(loss='mse',optimizer=adam_optimizer,metrics=[r2_score])
    model.summary()
    return model


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None, 100, 6)]             0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, 100, 128),           27648     ['Input[0][0]']               
 al)                          (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 concatenate (Concatenate)   (None, 128)                  0         ['bidirectional[0][1]',       
                                                                     'bidirectional[0][2]']   

In [10]:

file = open(f'data/minmax_SP500.pkl','rb')
minmax=pickle.load(file)

# Model Config
batch_size = 64
time_series = 100
num_features = 6
label_shape = (10,)
input_shape = (time_series,num_features)

meta_df = pd.read_csv('data/meta_SP500.csv',index_col=0)
train_meta, test_meta = train_test_split(meta_df, test_size=0.2, random_state=4211)
minmax=None
# display(train_meta)
# display(test_meta)
train_generator = DataGenerator(train_meta,input_shape=input_shape, batch_size=batch_size, minmax=minmax,label_shape=label_shape)
val_generator = DataGenerator(test_meta,input_shape=input_shape, batch_size=batch_size, minmax=minmax, use_full=True,label_shape=label_shape)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
x, y = train_generator.__getitem__(0)
print(x[0,:])
print(y[0, :])

[[ 0.00000000e+00  1.85434688e-02  1.57950656e-03  1.51310833e-02
   0.00000000e+00 -4.71137106e-01]
 [ 0.00000000e+00  2.45901924e-02  4.34384774e-03  1.64820533e-02
  -5.28473035e-03 -5.11968553e-01]
 [ 0.00000000e+00  9.27193277e-03  1.84302195e-03  1.40504697e-02
   5.02063986e-03 -4.82920676e-01]
 [ 0.00000000e+00  1.22279851e-02  2.63257039e-04  1.28344717e-02
  -3.96298012e-04  1.12423442e-01]
 [ 0.00000000e+00  1.85435042e-02 -6.58159843e-04  9.99740791e-03
  -1.84960698e-03 -3.28254521e-01]
 [ 0.00000000e+00  2.31121071e-02  3.55401402e-03  1.35109248e-03
   3.43501521e-03 -3.51506501e-01]
 [ 0.00000000e+00  1.69311166e-02 -5.26383752e-04  1.48610855e-02
  -5.68092009e-03 -6.31941676e-01]
 [ 0.00000000e+00  1.43778855e-02  3.31381003e-08  1.68873630e-02
  -4.49202629e-03 -3.59508365e-01]
 [ 0.00000000e+00  1.26310466e-02 -3.29077267e-03  1.05377799e-02
   0.00000000e+00 -5.24382412e-01]
 [ 0.00000000e+00  1.33029250e-02 -4.73874481e-03  7.02516222e-03
  -1.40045835e-02 -5.9146

In [45]:

# Directory Config
neural_path = 'neural_network'
save_path_dir='attention_t20_fullFeatures_encode_decode'

if not os.path.exists(f'{neural_path}/{save_path_dir}'):
    os.makedirs(f'{neural_path}/{save_path_dir}')

checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_r2_score',
    save_weights_only=False,
    save_best_only=True,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)

import pickle
file = open(f'data/scaler_SP500.pkl','rb')
scaler=pickle.load(file)
scaler=None
init_neuron=64
learning_rate=0.0002
model = build_model(time_series,num_features,init_neuron=init_neuron,learning_rate=learning_rate,scaler=scaler)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None, 100, 6)]             0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, 100, 128),           27648     ['Input[0][0]']               
 al)                          (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 concatenate (Concatenate)   (None, 128)                  0         ['bidirectional[0][1]',       
                                                                     'bidirectional[0][2]']   

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
history = model.fit(train_generator, batch_size=batch_size, epochs=2000, validation_data=val_generator, callbacks=[checkpoint, earlystop])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")


Epoch 1/2000
46/46 [==============================] - ETA: 0s - loss: 0.0336 - r2_score: 0.1986
Epoch 1: val_r2_score improved from -inf to 0.34388, saving model to neural_network/attention_t20_fullFeatures_encode_decode/saved_model.keras
46/46 [==============================] - 43s 678ms/step - loss: 0.0336 - r2_score: 0.1986 - val_loss: 0.0299 - val_r2_score: 0.3439
Epoch 2/2000
46/46 [==============================] - ETA: 0s - loss: 0.0309 - r2_score: 0.3633
Epoch 2: val_r2_score improved from 0.34388 to 0.37168, saving model to neural_network/attention_t20_fullFeatures_encode_decode/saved_model.keras
46/46 [==============================] - 37s 816ms/step - loss: 0.0309 - r2_score: 0.3633 - val_loss: 0.0299 - val_r2_score: 0.3717
Epoch 3/2000
46/46 [==============================] - ETA: 0s - loss: 0.0296 - r2_score: 0.3722
Epoch 3: val_r2_score improved from 0.37168 to 0.37775, saving model to neural_network/attention_t20_fullFeatures_encode_decode/saved_model.keras
46/46 [======

In [37]:
total_count = 0
correct_count = 0
uptrend_correct = 0 #Correct Uptrend
uptrend_wrong = 0
downtrend_correct = 0 #Correct Downtrend
downtrend_wrong = 0
for i in range(len(val_generator)):
   x, y_actual = val_generator.__getitem__(i)
   total_count += y_actual.shape[0] * y_actual.shape[1]
   y_pred = model.predict(x,verbose=0)
   y_actual_label = y_actual[:,:] >= 0.03
   y_pred_label = (y_pred[:,:].reshape(-1,10)) >= 0.03

   correct = y_actual_label == y_pred_label
   correct_count += np.count_nonzero(correct)
   uptrend_correct += np.count_nonzero(correct & y_pred_label)
   downtrend_correct += np.count_nonzero(correct & ~y_pred_label)
   uptrend_wrong += np.count_nonzero(~correct & y_pred_label)
   downtrend_wrong += np.count_nonzero(~correct & ~y_pred_label)
   break



print(f"Accuracy: {correct_count/total_count}")
print(f"Total: {total_count}")
print(f"Correct Count: {correct_count}")

print(f"Uptrend Total: {uptrend_correct+uptrend_wrong}")
print(f"Uptrend Correct: {uptrend_correct}")
print(f"Uptrend Accuracy: {uptrend_correct/(uptrend_correct+uptrend_wrong)}")

print(f"Downtrend Total: {downtrend_correct+downtrend_wrong}")
print(f"Downtrend Correct: {downtrend_correct}")
print(f"Downtrend Accuracy: {downtrend_correct/(downtrend_correct+downtrend_wrong)}")

print(x[0,-5:-1,-2])
print(y_actual[0,:])
print(y_pred[0,:])


Accuracy: 0.878125
Total: 640
Correct Count: 562
Uptrend Total: 349
Uptrend Correct: 302
Uptrend Accuracy: 0.8653295128939829
Downtrend Total: 291
Downtrend Correct: 260
Downtrend Accuracy: 0.8934707903780069
[0.13392295 0.19650632 0.17392506 0.16102105]
[0.2048939  0.20876486 0.23489508 0.16585988 0.13359996 0.19618397
 0.20166785 0.21812033 0.20027526 0.20842275]
[[0.17631081]
 [0.18597493]
 [0.18941073]
 [0.19809292]
 [0.20914519]
 [0.20593041]
 [0.21678111]
 [0.22929043]
 [0.23472734]
 [0.24474347]]


In [14]:
profit_count = 0
trade_count = 0
total_samples = 0
change = 0
start_money=0
per_trade = 1000
def backtest():
  global trade_count, profit_count, change,start_money, per_trade, total_samples
  for i in range(len(val_generator)):
    x, y_actual = val_generator.__getitem__(i)
    start_value = x[:,-1,-2] # Close Price
    y_pred = model.predict(x,verbose=0)

    for i in range(x.shape[0]):
      y_pred_idx = np.argmax(y_pred,axis=1)[i]

      x_start = start_value[i] + 1
      y_pred_stop = y_pred[i, y_pred_idx].reshape(-1)[0] + 1
      y_actual_stop = y_actual[i, y_pred_idx][0] + 1

      #Assumption: Buy Same value of stock -> Percentage Change is equally scale
      if y_pred_stop > x_start: # Predict Profitable, Buy
        change_value = (y_actual_stop - x_start)/x_start
        if change_value > 0:
          profit_count+=1

        trade_count+=1
        print(f'start: {x_start} end: {y_actual_stop} change: {change_value} idx: {y_pred_idx}')
        change+= change_value
        start_money = start_money + per_trade*change_value
      total_samples+=1


  print(f"Total Trade: {trade_count}")
  print(f"Profitable Trade: {profit_count}")
  print(f"Accuracy: {profit_count/trade_count}")
  print(f"Net Change: {change}")
  print(f"Final Income: {start_money}")
  print(f"Total Samples: {total_samples}")
  print(f"Trade Coverage: {trade_count/total_samples}")
backtest()

start: 1.01419158000499 end: 1.1343501548867836 change: 0.11847719627213063 idx: [9]
start: 1.118441253900528 end: 1.1652828571968352 change: 0.04188114765344061 idx: [9]
start: 1.1094694510102272 end: 1.1044324590763885 change: -0.004540000564460997 idx: [9]
start: 1.0767659395933151 end: 1.0335720035706608 change: -0.04011450811581974 idx: [9]
start: 1.3853653967380524 end: 1.090212046285884 change: -0.21305090422146336 idx: [9]
start: 0.8981917276978493 end: 0.9053129357530699 change: 0.007928383034068889 idx: [7]
start: 1.1655543744564056 end: 1.155008185493165 change: -0.00904821704964139 idx: [9]
start: 1.3185445070266724 end: 1.4144775891172379 change: 0.07275680235238727 idx: [9]
start: 1.0900290459394455 end: 1.0014310001623226 change: -0.08128044487178265 idx: [9]
start: 0.9874774795025587 end: 0.9715197860246353 change: -0.016160058137186174 idx: [8]
start: 1.1704782396554947 end: 1.1546457437665416 change: -0.013526518778865156 idx: [9]
start: 1.027886526659131 end: 1.14062